### Read Data and Clean

In [19]:
import pandas as pd
import numpy as np
import boto3, re, sys, math, json, os, sagemaker, urllib.request, IPython
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pickle

In [28]:
from sklearn.linear_model import LinearRegression

In [20]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [21]:
if int(sagemaker.__version__.split('.')[0]) == 2:
    print("Installing previous SageMaker Version and restarting the kernel")
    !{sys.executable} -m pip install sagemaker==1.72.0
    IPython.Application.instance().kernel.do_shutdown(True)

else:
    print("Version is good")

Version is good


In [22]:
role = get_execution_role()
sess = sagemaker.Session()
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')
# create a s3 bucket to store model related stuff
bucket_name = 'airplane-ticket-model'
s3 = boto3.resource('s3')
try:
    if  region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

Region = us-east-1
S3 bucket created successfully


In [23]:
data = pd.read_csv('data/Cleaned_2018_Flights.csv')
data = data.dropna()
data = data.drop(
    columns=[
        "Unnamed: 0",
        "ItinID",
        "MktID",
        "MktCoupons",
        "OriginWac",
        "DestWac",
        "ContiguousUSA",
        "Miles",
        "PricePerTicket"
    ]
)

In [24]:
data.head()

,Quarter,Origin,Dest,NumTicketsOrdered,AirlineCompany
0,1,PHL,LAX,1.0,AA
1,1,PHL,LAX,1.0,AA
2,1,PHL,LAX,1.0,AA
3,1,PHL,LAX,1.0,AA
4,1,PHL,LAX,1.0,AA


In [25]:
numerical_cols = ["NumTicketsOrdered"]
categorical_cols = ["Quarter", "Origin", "Dest", "AirlineCompany"]

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, categorical_cols)],
    remainder="passthrough",
)

In [26]:
preprocessor.fit(data)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 Pipeline(memory=None,
                                          steps=[('onehot',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='ignore',
                                                                sparse=True))],
                                          verbose=False),
                                 ['Quarter', 'Origin', 'Dest',
                                  'AirlineCompany'])],
                  verbose=False)

In [27]:
transformed_data = preprocessor.transform(data)

In [33]:
data = pd.read_csv('data/Cleaned_2018_Flights.csv')
y = data[["PricePerTicket"]]

In [34]:
lr = LinearRegression()

In [36]:
lr.fit(transformed_data, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [37]:
file = "lr_model.sav"
pickle.dump(lr, open(file, 'wb'))

In [11]:
file = "preprocerssor.sav"
pickle.dump(preprocessor, open(file, 'wb'))

In [12]:
mod = pickle.load(open(file, 'rb'))

In [14]:
mod.transform(data.head()).toarray()

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [38]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file, bucket_name, "lr_model.sav")